<a href="https://colab.research.google.com/github/GRAY-DDOT/family_face/blob/main/getFacesWithFID.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 열람 및 실행 전 참고사항
# *개인 정보 보호를 위해 출력된 결과 일부를 오류로 덮어씌움
# *정상적인 결과 출력을 위해서는 다음사항에 대한 수정이 필요함
# 1. shape_predictor 업로드
# 2. 시트 데이터
# 3. 시트에 따른 코드 수정
# 4. path, os.makedir 관련 설정

In [ ]:

import cv2
import os
import glob
import gspread

import numpy as np
import pandas as pd
import json
import dlib
import urllib.request
from google.colab.patches import cv2_imshow

# 랜드마크경로
predictor = dlib.shape_predictor("프리딕터 위치")
detector = dlib.get_frontal_face_detector()

#url => cv2.imdecode(cv2.IMREAD_COLOR)
def url_to_image(url):
  resp = urllib.request.urlopen(url)
  image = np.asarray(bytearray(resp.read()), dtype='uint8')
  image = cv2.imdecode(image, cv2.IMREAD_COLOR)

  return image

# 1. 파일 준비
## 1.1. 정보 준비
### gspread 접근

https://docs.gspread.org/en/latest/

### gspread Authentication

In [ ]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

### gspread load

In [ ]:
# 스프레드 주소
sheetUrl = "시트 URL"
# 시트에는 가족별 사진 제출 타입(제출 장 수), 사진 위치(디렉토리 또는 사진 URL https://drive.google.com/uc?id= 과 같이 다운로드 가능한 주소이어야함.)
sh = gc.open_by_url(sheetUrl)

In [ ]:
#시트 접근 및 워크페이지 저장(0번째 시트 => 가족 정보 및 변환 url 저장)
info = sh.get_worksheet(0)


In [ ]:
#시트 데이터 전부 불러오기
info_all = info.get_all_values()

In [ ]:
#시트 데이터 판다스 데이터 프레임으로 변환
df_info_all = pd.DataFrame(info_all)
df_info_all = df_info_all.rename(columns={0:'family_ID', 1:'f_age', 2:'m_age',3:'c1_age',4:'c1_gender',5:'c2_age',6:'c2_gender',7:'c3_age',8:'c3_gender',9:'type',10:'3iP',11:'inF',12:'inM',13:'inC',14:'duo',15:'solo'})

df_info_all

## 2. 가족별 폴더 생성 및 json 작성

In [ ]:
#Family_Face_Public/prcessed
#path_for_not_aligned
#path= "저장할 디렉토리/"
#path_for_aligned
path= "저장할 디렉토리/"

In [ ]:
f_id_list=df_info_all['family_ID'].tolist()

print(f_id_list)
print(type(f_id_list))

In [ ]:
c1g_list = df_info_all['c1_gender'].tolist()
c2g_list = df_info_all['c2_gender'].tolist()
c3g_list = df_info_all['c3_gender'].tolist()
print(c1g_list)
print(c2g_list)
print(c3g_list)

In [ ]:
for i in f_id_list:
  os.makedirs(path+'생성하고자 하는 디렉토리 이름 및 로직'+"/", exist_ok = True)


In [ ]:
# JSON 작성
for i in f_id_list:
  #자식 성별 로드
  cg1 = c1g_list[int(i)-1]
  cg2 = c2g_list[int(i)-1]
  cg3 = c3g_list[int(i)-1]

  # JSON 데이터 정의


  if(cg2 == ''):
    print("cg1 = " +cg1,"  \tcg2 = None","  \tcg3 = None")
    jsonData = {
        "family_id": 처리 로직,
        "num_image": 3,
        "num_family_member": 3,
        "family_list":["F", "M", cg1 ]
    }
  elif(cg2 != ''):
    if(cg3 == ''):
      print("cg1 = " +cg1,"  \tcg2 = "+cg2,"  \tcg3 = None")
      jsonData = {
        "family_id": 처리 로직,
        "num_image": 4,
        "num_family_member": 4,
        "family_list":["F", "M", cg1, cg2]
      }
    elif(cg3 != ''):
      print("cg1 = " +cg1,"  \tcg2 = "+cg2,"  \tcg3 = "+cg3)
      jsonData = {
        "family_id": 처리 로직,
        "num_image": 5,
        "num_family_member": 5,
        "family_list":["F", "M", cg1, cg2, cg3]
      }
  # JSON 파일로 저장

  with open(path+"이름 작성 로직", 'w') as json_file:
    json.dump(jsonData, json_file, indent=4)


# 3. 이미지 가공

## 이미지 디렉토리 조건 설정

In [ ]:
## 이미지 디렉토리 조건 설정
img_type_list = df_info_all['type'].tolist()
print(img_type_list)
print(type(img_type_list))

print(f_id_list)
print(type(f_id_list))

In [ ]:
#가족 id 대로 입력
#family_ID	f_birth	m_birth	c_birth	c_gender	type	solo_role	3iP	inF	inM	inC	duo	solo
*개인 정보 보호를 위해 출력된 결과 일부를 오류로 덮어씌움
*정상적인 결과 출력을 위해서는 다음사항에 대한 수정이 필요함
#1. shape_predictor 업로드
#2. 시트 데이터
#3. 시트에 따른 코드 수정
#4. path, os.makedir 관련 설정
for i in range(len(f_id_list)):
  img_type = img_type_list[i]

  print("=======================================================")
  print("["+str(i)+"]")
  print("img_type= "+img_type)

  dir= []
  print("sizeof dir= "+str(len(dir)))
  if(img_type == 'a'):
    dir.insert(0,df_info_all['3iP'][i])
    print("Validated")
    print(dir)
  elif(img_type == 'c'):
    dir.insert(0, df_info_all['duo'][i])
    dir.insert(1, df_info_all['solo'][i])
    print("Validated")
    print(dir)
  elif(img_type == 'b'):
    dir.insert(0, df_info_all['inF'][i])
    dir.insert(1, df_info_all['inM'][i])
    dir.insert(2, df_info_all['inC'][i])
    print("Validated")
    print(dir)
  elif(img_type == 'None'):
    print("continue-i-img_type == 'None")
    continue

  else: print("expception: invalide img_type")

  print("sizeof dir= "+str(len(dir)))

  l=0
  for j in dir:
    print(j)
    l+=1
    if(img_type == 'None'):
      print("[break-j-img_type == 'None]'")
      break


    img = url_to_image(j)


    #aligned
    dets = detector(img, 1)


    # Find the 5 face landmarks we need to do the alignment.
    faces = dlib.full_object_detections()
    for detection in dets:
        faces.append(predictor(img, detection))



    # Get the aligned face images
    # Optionally:
    # images = dlib.get_face_chips(img, faces, size=160, padding=0.25)

    images = dlib.get_face_chips(img, faces, size = 112)
    for k in range(len(images)):
      output_filename = path+"이름 작명 로직"+'.jpg'
      cv2_imshow(images[k])
      cv2.imwrite(output_filename, images[k])

    """
    #not aligned(미정렬, 얼굴 영역 크게)
    dets = detector(img, 1)
    # Loop through all detected faces
    for k, d in enumerate(dets):
      # Get facial landmarks for the current face
      shape = predictor(img, d)

      # Define the parameters for drawing landmarks and annotations (similar to your existing code)
      top_y =d.top()
      bottom_y = d.bottom()
      left_x = d.left()
      right_x = d.right()

      width = right_x - left_x
      height = top_y - bottom_y

      center_y = int(0.5*(top_y + bottom_y))
      center_x = int(0.5*(left_x+ right_x))

      top_y = center_y + int(1.0*height)
      bottom_y = center_y - int(0.6*height)
      left_x = center_x - int(0.8*width)
      right_x = center_x + int(0.8*width)


      # Modify the top part of the rectangle to extend it

      # 정사각형 크기 결정 (너비와 높이 중 큰 값 사용)
      square_size = max(right_x - left_x, bottom_y - top_y)

      # 영역 경계 조정 (이미지 경계를 넘지 않도록)
      top = max(center_y - square_size // 2, 0)
      bottom = min(center_y + square_size // 2, img.shape[0])
      left = max(center_x - square_size // 2, 0)
      right = min(center_x + square_size // 2, img.shape[1])

      if((right -left)!=(bottom - top)):
        if ((right -left)>(bottom - top)):
          right -= 1
        else:
          bottom -= 1


      # Crop and resize the current face

      crop = img[top:bottom, left:right]


      cv2_imshow(crop)

      # Save or display the cropped and resized face
      # You can save each cropped face with a unique filename
      output_filename = path+"이름 작명 로직"+'.jpg'
      cv2.imwrite(output_filename, crop)
      print("img saved!")


      # Draw a rectangle around the detected face (commented out to remove rectangle)
      # cv2.rectangle(img, (left_x, top_y), (right_x, bottom_y), (0, 255, 0), 2)
      """
